# Backtesting Tutorial
Del video: https://www.youtube.com/watch?v=38axuBMgLwA&t=189s&ab_channel=SeriousBacktester 
Aqui se explica una forma básica de implementar una estrategia llamada "Wick Rejection", que es de autoría de Trader Pro ( https://www.youtube.com/watch?v=zCWvwZO4_fE&ab_channel=TradePro )


La estrategia consta de las siguientes normas:
- 200 EMA
- 50 EMA
- ATR (14 periodos)

Estrategia: 
LONG: Se busca que el precio esté por encima de la EMA200, se compra cunado el precio llega a romper la EMA50 (se espera uin rebote)
Entry: El precio cruza la EMA50 (el LOW del dia) PERO tiene que cerrar por encima de la EMA50 (CLOSE > 50EMA). La entrada es el OPEN de la vela siguiente.
TP: 1.5R (fixed TP) = Entry + ATR
SL: Precio de entrada - ATR 

In [1]:
import pandas as pd
import yfinance as yf
from finta import TA


In [2]:
df = yf.download(['AAPL'], period='5y', auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2018-04-23,39.760635,39.782084,39.107609,39.381691,146062000
2018-04-24,39.484183,39.641481,38.423613,38.833542,134768000
2018-04-25,38.757267,39.424592,38.707219,39.002747,113528400
2018-04-26,39.114773,39.498485,38.936025,39.138607,111852000
2018-04-27,39.086173,39.164822,38.283001,38.685780,142623200
...,...,...,...,...,...
2023-04-17,165.089996,165.389999,164.029999,165.229996,41516200
2023-04-18,166.100006,167.410004,165.649994,166.470001,49923000
2023-04-19,165.800003,168.160004,165.539993,167.630005,47720200


Definimos los indicadores tecnicos necesarios.

In [4]:
df["EMA200"] = TA.EMA(df, period=200)
df["EMA50"] = TA.EMA(df, period=50)
df["ATR"] = TA.ATR(df)


In [5]:
df

,Open,High,Low,Close,Volume,EMA200,EMA50,ATR
Date,,,,,,,,
2018-04-23,39.760635,39.782084,39.107609,39.381691,146062000,39.381691,39.381691,NaN
2018-04-24,39.484183,39.641481,38.423613,38.833542,134768000,39.106246,39.102135,NaN
2018-04-25,38.757267,39.424592,38.707219,39.002747,113528400,39.071401,39.067672,NaN
2018-04-26,39.114773,39.498485,38.936025,39.138607,111852000,39.088455,39.086483,NaN
2018-04-27,39.086173,39.164822,38.283001,38.685780,142623200,39.006301,38.999806,NaN
...,...,...,...,...,...,...,...,...
2023-04-17,165.089996,165.389999,164.029999,165.229996,41516200,150.159576,155.767544,2.825715
2023-04-18,166.100006,167.410004,165.649994,166.470001,49923000,150.321869,156.187249,2.802144
2023-04-19,165.800003,168.160004,165.539993,167.630005,47720200,150.494090,156.635984,2.746429


Creamos las condiciones de la estrategia. Recordemos:
Estrategia: 
- LONG: Se busca que el precio esté por encima de la EMA200, se compra cunado el precio llega a romper la EMA50 (se espera uin rebote)
- Entry: El precio cruza la EMA50 (el LOW del dia) PERO tiene que cerrar por encima de la EMA50 (CLOSE > 50EMA). La entrada es el OPEN de la vela siguiente.
- TP: 1.5R (fixed TP) = Entry + ATR
- SL: Precio de entrada - ATR 

In [6]:
# Creamos las condiciones

c1 = df["Close"] > df["EMA200"]
c2 = (df["Low"] < df["EMA50"]) & (df["Close"] > df["EMA50"]) & (df["Open"] > df["EMA50"])
long_entry_condition = (c1) & (c2)

In [7]:
# Al hacer este loc me da los dias en los que la condición para la entrada se cumplió
# Cómo hace esto? es una especie de filtro o k???? no me queda claro
df.loc[long_entry_condition]


,Open,High,Low,Close,Volume,EMA200,EMA50,ATR
Date,,,,,,,,
2018-04-26,39.114773,39.498485,38.936025,39.138607,111852000,39.088455,39.086483,NaN
2018-10-12,52.920281,53.510900,52.060764,53.326031,161351600,49.287263,52.165908,1.298020
2018-10-15,53.097948,53.258807,52.164005,52.185612,123164000,49.328013,52.166687,1.327516
2018-10-16,52.562547,53.537309,52.041556,53.335632,116736000,49.384128,52.212851,1.365931
2018-10-19,52.353679,53.121961,52.202422,52.653790,132314800,49.513798,52.249524,1.455277
2018-11-01,52.591360,53.386052,52.053561,53.352440,233292800,49.837961,52.265751,1.654553
2019-05-10,47.951575,48.298912,46.822133,47.893280,164834800,45.464510,47.183174,1.269901
2019-08-09,49.079759,49.435725,48.589690,49.004177,98478800,46.671651,48.660187,1.304729
2019-08-12,48.670157,49.262627,48.555564,48.879837,89927600,46.694473,48.668801,1.325913


Esto ya me funcinona, si validas los puntos de entrada corresponden a lo que debe ser
sin embargo, sería mejor que pudieramos tambien tener el SL y TP para poder tirar numeros

In [8]:
# Shift entiendo que te desplaza la data en este caso 1: 
df["entry"] = df.shift(-1)["Open"]

In [9]:
df

,Open,High,Low,Close,Volume,EMA200,EMA50,ATR,entry
Date,,,,,,,,,
2018-04-23,39.760635,39.782084,39.107609,39.381691,146062000,39.381691,39.381691,NaN,39.484183
2018-04-24,39.484183,39.641481,38.423613,38.833542,134768000,39.106246,39.102135,NaN,38.757267
2018-04-25,38.757267,39.424592,38.707219,39.002747,113528400,39.071401,39.067672,NaN,39.114773
2018-04-26,39.114773,39.498485,38.936025,39.138607,111852000,39.088455,39.086483,NaN,39.086173
2018-04-27,39.086173,39.164822,38.283001,38.685780,142623200,39.006301,38.999806,NaN,38.640488
...,...,...,...,...,...,...,...,...,...
2023-04-17,165.089996,165.389999,164.029999,165.229996,41516200,150.159576,155.767544,2.825715,166.100006
2023-04-18,166.100006,167.410004,165.649994,166.470001,49923000,150.321869,156.187249,2.802144,165.800003
2023-04-19,165.800003,168.160004,165.539993,167.630005,47720200,150.494090,156.635984,2.746429,166.089996


In [10]:
# Creamos columnas en 0 para el TP y SL¿
df["target"] = 0.0
df["stop"] = 0.0

In [11]:
# Aplicamos las condiciones para encontrar los puntos de entrada y salida

df.loc[long_entry_condition, "stop"] = df["entry"] - df["ATR"]
df.loc[long_entry_condition, "target"] = df["entry"] + 1.5*df["ATR"]


In [12]:
df.loc[long_entry_condition]

,Open,High,Low,Close,Volume,EMA200,EMA50,ATR,entry,target,stop
Date,,,,,,,,,,,
2018-04-26,39.114773,39.498485,38.936025,39.138607,111852000,39.088455,39.086483,NaN,39.086173,NaN,NaN
2018-10-12,52.920281,53.510900,52.060764,53.326031,161351600,49.287263,52.165908,1.298020,53.097948,55.044979,51.799928
2018-10-15,53.097948,53.258807,52.164005,52.185612,123164000,49.328013,52.166687,1.327516,52.562547,54.553822,51.235031
2018-10-16,52.562547,53.537309,52.041556,53.335632,116736000,49.384128,52.212851,1.365931,53.371643,55.420539,52.005712
2018-10-19,52.353679,53.121961,52.202422,52.653790,132314800,49.513798,52.249524,1.455277,52.769030,54.951946,51.313752
2018-11-01,52.591360,53.386052,52.053561,53.352440,233292800,49.837961,52.265751,1.654553,50.310522,52.792351,48.655970
2019-05-10,47.951575,48.298912,46.822133,47.893280,164834800,45.464510,47.183174,1.269901,45.593100,47.497952,44.323198
2019-08-09,49.079759,49.435725,48.589690,49.004177,98478800,46.671651,48.660187,1.304729,48.670157,50.627250,47.365428
2019-08-12,48.670157,49.262627,48.555564,48.879837,89927600,46.694473,48.668801,1.325913,49.011493,51.000362,47.685581


Ahora que ya tenemos la entrada y la salida, podemos crear una función que pueda aceptar cualquier stock y te regrese ese df

In [13]:
def wick_rejection(ticker):

    # Creamos el df descargando la data del ticker
    df = yf.download([ticker], period='5y', auto_adjust=True)

    # Indicadores teçnicos
    df["EMA200"] = TA.EMA(df, period=200)
    df["EMA50"] = TA.EMA(df, period=50)
    df["ATR"] = TA.ATR(df)

    # Corremos 1 el df para definir la entrada de la posicion
    df["entry"] = df.shift(-1)["Open"]

    # Shift to find entry point
    df["entry"] = df.shift(-1)["Open"]

    # Creamos columnas en 0 para el TP y SL¿
    df["target"] = 0.0
    df["stop"] = 0.0

    # Find entry and exit
    df.loc[long_entry_condition, "stop"] = df["entry"] - df["ATR"]
    df.loc[long_entry_condition, "target"] = df["entry"] + 1.5*df["ATR"]
    df.loc[long_entry_condition]

    print(f'Hay {df.loc[long_entry_condition].shape[0]} entradas para el ticker {ticker}')
    return df.loc[long_entry_condition]
    


wick_rejection("MSFT")




[*********************100%***********************]  1 of 1 completed
Hay 19 entradas para el ticker MSFT


,Open,High,Low,Close,Volume,EMA200,EMA50,ATR,entry,target,stop
Date,,,,,,,,,,,
2018-04-26,88.290126,89.800164,87.865423,88.960205,42529000,88.483550,88.469857,NaN,92.112413,NaN,NaN
2018-10-12,103.724433,105.846303,101.926075,104.257278,47742100,101.166009,104.883653,2.454902,103.629276,107.311629,101.174375
2018-10-15,103.629276,104.171639,101.764305,102.382790,32068100,101.183116,104.784860,2.541218,104.228742,108.040570,101.687524
2018-10-16,104.228742,106.008074,103.667346,105.617950,31610200,101.245214,104.817760,2.710453,106.264979,110.330659,103.554526
2018-10-19,103.648301,105.484721,102.963210,103.391396,32785500,101.357781,104.720721,3.027173,104.019402,108.560162,100.992230
2018-11-01,101.859470,102.116376,100.413166,100.784256,33384200,101.317559,103.575613,3.834598,101.317103,107.069000,97.482505
2019-05-10,119.876732,122.775038,118.830650,122.007271,30915100,107.581250,115.916564,2.500025,119.108971,122.859008,116.608946
2019-08-09,133.517057,134.258769,131.446060,132.650131,23466700,118.055543,129.740176,2.991600,132.033672,136.521072,129.042073
2019-08-12,132.033672,132.794639,130.270910,130.800690,20484300,118.187268,129.781765,3.037010,131.051198,135.606712,128.014188
